In [ ]:
import pandas as pd
import requests
import os
import zipfile

diretorio_atual = os.getcwd()

os.chdir(f"{diretorio_atual}/dados_cvm")

In [ ]:
anos = range(2010, 2023)

url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

for ano in anos:
    
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

In [ ]:
lista_demontracoes_2010_2022 = []

diretorio_atual = os.getcwd()

for arquivo in os.listdir(diretorio_atual):
    
    if arquivo == ".ipynb_checkpoints":
        
        pass
    
    else:
        
        arquivo_zip = zipfile.ZipFile(arquivo)
        
        for planilha in arquivo_zip.namelist():
            
            demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep = ";", encoding = 'ISO-8859-1',
                                      dtype = {"ORDEM_EXERC": "category"})
            
            lista_demontracoes_2010_2022.append(demonstracao)
            


In [ ]:
base_dados = pd.concat(lista_demontracoes_2010_2022)

base_dados

In [ ]:
base_dados[['con_ind', 'tipo_dem']] = base_dados['GRUPO_DFP'].str.split("-", expand = True)
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()

base_dados = base_dados[base_dados['ORDEM_EXERC'] != "PENÚLTIMO"]

In [ ]:
lista_dem = base_dados['tipo_dem'].unique()

lista_dem

In [ ]:
lista_empresas = base_dados['DENOM_CIA'].unique()

lista_empresas

In [ ]:
weg_dre = base_dados[(base_dados["DENOM_CIA"] == "WEG S.A.") & 
                     (base_dados["tipo_dem"] == "Demonstração do Resultado") &
                    (base_dados["DS_CONTA"] == "Receita de Venda de Bens e/ou Serviços") &
                    (base_dados["con_ind"] == "DF Consolidado")]

weg_dre = weg_dre[['DT_REFER', 'VL_CONTA']]

weg_dre

In [12]:
weg_dre

,DT_REFER,VL_CONTA
22814,2010-12-31,4391973.0
23721,2011-12-31,5189409.0
23379,2012-12-31,6173878.0
22975,2013-12-31,6828896.0
22612,2014-12-31,7840757.0
22410,2015-12-31,9760323.0
22497,2016-12-31,9367008.0
22381,2017-12-31,9523830.0
22209,2018-12-31,11970090.0
24766,2019-12-31,13347434.0
